In [1]:
import pandas as pd
from engine.hyperparameters_tune_raytune import deepscreen_hyperparameter_tuneing
from utils.configurations import configs
import os

/home/sjinich/disco/che_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-23 17:32:20,278	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-23 17:32:20,441	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-23 17:32:22,194	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import warnings
warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argumen*")

data = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2581.csv")
    
search_space_deepscreen = configs.get_hyperparameters_search()

tuner = deepscreen_hyperparameter_tuneing(
    data=data,
    data_split_mode="non_random_split",
    search_space=search_space_deepscreen,
    target="chembl2581_trial",
    experiments_result_path="../../.experiments",
    **configs.get_hyperparameters_search_setup()
    )

result = tuner.tune_deepscreen()


(TorchTrainer pid=1319480) Started distributed worker processes: 
(TorchTrainer pid=1319480) - (ip=10.1.103.91, pid=1320314) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=1320262) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=1320262) INFO: Using hyperparameters [('fully_layer_1', 128), ('fully_layer_2', 512), ('drop_rate', 0.3), ('learning_rate', 0.005), ('batch_size', 32), ('experiment_result_path', '/big/lab/sjinich/TrypanoDEEPscreen/.experiments/chembl2581_trial')]
(RayTrainWorker pid=1320262) GPU available: False, used: False
(RayTrainWorker pid=1320262) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=1320262) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=1320262) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=1320262) [rank: 0] Seed set to 123
(RayTrainWorker pid=1320262) Missing logger folder: /home/sjinich/ray_results/TorchTrainer_2024-02-23_17-32-22/TorchTrainer_823bd_00000_0_batch_size=32,drop_rat

In [11]:
ckpt_dir = os.path.join(result.get_best_result(metric="val_mcc",mode="max").checkpoint.path,"checkpoint.ckpt")

In [12]:
from engine.system import DEEPScreenClassifier
from datasets.datamodule import DEEPscreenDataModule

model = DEEPScreenClassifier.load_from_checkpoint(ckpt_dir,experiment_result_path="../../.experiments/chembl2581")

INFO: Using hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.3), ('learning_rate', 0.0001), ('batch_size', 32), ('experiment_result_path', '../../.experiments/chembl2581')]


In [15]:
data = pd.read_csv("../../.data/processed/CHEMBL2581.csv")
datamodule = DEEPscreenDataModule(data=data,target_id="CHEMBL2581",batch_size=32,experiment_result_path="../../.experiments/chembl2581",data_split_mode="non_random_split",tmp_imgs=True)


In [16]:
from lightning import Trainer

trainer = Trainer()

trainer.test(model,datamodule=datamodule)

GPU available: False, used: False


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO: Using a total of 2270 datapoints
INFO: non_random_split datasets splited train=1452,validation=364,test=454


Testing DataLoader 0:  60%|██████    | 9/15 [00:02<00:01,  3.40it/s]

/home/sjinich/disco/che_env/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:211: You called `self.log('test_mcc', ...)` in your `test_step` but the value needs to be floating point. Converting it to torch.float32.


Testing DataLoader 0: 100%|██████████| 15/15 [00:04<00:00,  3.59it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.7819383144378662
         test_f1            0.8176795840263367
        test_loss           0.4834350347518921
        test_mcc             0.555237352848053
        test_prec           0.7708333134651184
       test_recall          0.8705882430076599
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.4834350347518921,
  'test_acc': 0.7819383144378662,
  'test_f1': 0.8176795840263367,
  'test_mcc': 0.555237352848053,
  'test_prec': 0.7708333134651184,
  'test_recall': 0.8705882430076599}]

In [20]:
result_df = result.get_dataframe()
result_df.loc[2]

train_loss                                           0.454345
val_loss                                             0.637153
val_acc                                              0.604396
val_f1                                               0.566265
val_mcc                                              0.356962
val_prec                                             0.895238
val_recall                                           0.414097
train_acc                                            0.760128
train_f1                                             0.765807
train_mcc                                            0.526185
train_prec                                           0.751276
train_recall                                         0.789661
epoch                                                       2
step                                                       69
timestamp                                          1708710088
checkpoint_dir_name                         checkpoint_000002
should_c

In [26]:
dir(result.get_best_result(metric="val_mcc",mode="max").checkpoint)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__fspath__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotated',
 '_get_temporary_checkpoint_dir',
 '_uuid',
 'as_directory',
 'filesystem',
 'from_directory',
 'get_metadata',
 'path',
 'set_metadata',
 'to_directory',
 'update_metadata']

In [39]:
result.get_best_result(metric="val_mcc",mode="max").config["train_loop_config"]["batch_size"]

{'fully_layer_1': 256,
 'fully_layer_2': 32,
 'learning_rate': 0.0001,
 'batch_size': 32,
 'drop_rate': 0.3}